## Gesture detection

Be sure to be in the root directory of the project before running the following code cells.  

## Model loading

In [ ]:
# Import all the required libraries
import os
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import numpy as np

# Utility definitions
WORKSPACE_PATH = 'Tensorflow/workspace'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'
actionMap = {
    "SelectPrevious" : ("LeftL", "RIndexUp"),
    "SelectNext" : ("LeftL", "RIndexDown"),
    "SelectChild" : ("LeftL", "RIndexRight"),
    "SelectParent" : ("LeftL", "RIndexLeft"),

    "MoveZPositive" : ("LeftA", "RIndexUp"),
    "MoveZNegative" : ("LeftA", "RIndexDown"),
    "MoveXPositive" : ("LeftA", "RIndexRight"),
    "MoveXNegative" : ("LeftA", "RIndexLeft"),
    "MoveYPositive" : ("LeftA", "RIndexFront"),
    "MoveYNegative" : ("LeftA", "RThumbBack"),
    "MoveClear" : ("LeftA", "RClearC"),

    "RotateZPositive" : ("LeftO", "RIndexUp"),
    "RotateZNegative" : ("LeftO", "RIndexDown"),
    "RotateXPositive" : ("LeftO", "RIndexRight"),
    "RotateXNegative" : ("LeftO", "RIndexLeft"),
    "RotateYPositive" : ("LeftO", "RIndexFront"),
    "RotateYNegative" : ("LeftO", "RThumbBack"),
    "RotateClear" : ("LeftO", "RClearC"),

    "TransformGlobal" : ("LeftV", "RIndexUp"),
    "TransformLocal" : ("LeftV", "RIndexDown"),
    "ViewNext" : ("LeftV", "RIndexRight"),
    "ViewPrevious" : ("LeftV", "RIndexLeft"),

    "HideCurrent" : ("LeftOpenHand", "RIndexRight"),
    "UnhideCurrent" : ("LeftOpenHand", "RIndexLeft"),
    "UnhideAll" : ("LeftOpenHand", "RIndexUp"),
             }

Be sure to change the _checkpoint\_name_ variable to the name of your chosen model's checkpoint name.  

In [ ]:
checkpoint_name = 'ckpt-41'

# Load the pipeline.config file and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore the specified checkpoint (it must match an existing model)
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, checkpoint_name)).expect_partial()

# Computes the detections from the predictive model
@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections